# Dia 1 - Importação de Dados

In [33]:
# Importando bibliotecas

!pip install pyarrow
!pip install pandas

In [34]:
import pandas as pd
import glob

### Criando repositórios para os arquivos

In [101]:
# Criando diretórios e pastas para receber os arquivos do GitHub

from pathlib import Path

# Cria um caminho (sem criar ainda no sistema de arquivos)
pasta_emprestimos = Path("dados/emprestimos")
pasta_exemplares = Path("dados/exemplares")

# Criação das pastas, incluindo as intermediárias se não existirem
pasta_emprestimos.mkdir(parents=True, exist_ok=True)
pasta_exemplares.mkdir(parents=True, exist_ok=True)

### Realizando a coleta de dados

In [36]:
# Baixando os arquivos para as pastas correspondentes

import requests

# Baixando CSVs

# Lista de anos para concatenar o nome do arquivo.
# Os arquivos CSV tem o mesmo prefixo mudando apenas o final.
# Para não importar os arquivos um a um, cria-se uma lista com o nome final,
# e concatena com o prefixo, realizando o download com um loop
lista_emprestimos = []

# Criando a lista automáticamente
for ano in range(2010, 2021):
    for semestre in [1, 2]:
        if ano == 2020 and semestre == 2:
            continue # pula o segundo semestre de 2020 e encerra o loop
        lista_emprestimos.append(f'{ano}{semestre}')

# Loop de download
for ano in lista_emprestimos:
    # passando o link para download e concatenando o ano ao final do nome do arquivo
    url = f"https://raw.githubusercontent.com/franciscofoz/7_Days_of_Code_Alura-Python-Pandas/main/Dia_1-Importando_dados/Datasets/dados_emprestimos/emprestimos-{ano}.csv"
    
    # faz uma requisição HTTP e armazena (get) a resposta em r
    r = requests.get(url)
    
    # armazenando o arquivo na pasta correspondente
    with open(f"dados/emprestimos/emprestimos-{ano}.csv", "wb") as f:
        f.write(r.content) # escreve o conteúdo de "r", que é o que ele pegou do link, o arquivo.

# Baixando o Parquet

parquet_url = f"https://github.com/franciscofoz/7_Days_of_Code_Alura-Python-Pandas/raw/main/Dia_1-Importando_dados/Datasets/dados_exemplares.parquet"
r = requests.get(parquet_url)

with open("dados/exemplares/dados_exemplares.parquet", "wb") as f:
    f.write(r.content)

In [37]:
# Criando uma lista com os caminhos de todos os arquivos .csv na pasta emprestimos
caminhos_csv = glob.glob("dados/emprestimos/*.csv")
print(caminhos_csv)

['dados/emprestimos\\emprestimos-20101.csv', 'dados/emprestimos\\emprestimos-20102.csv', 'dados/emprestimos\\emprestimos-20111.csv', 'dados/emprestimos\\emprestimos-20112.csv', 'dados/emprestimos\\emprestimos-20121.csv', 'dados/emprestimos\\emprestimos-20122.csv', 'dados/emprestimos\\emprestimos-20131.csv', 'dados/emprestimos\\emprestimos-20132.csv', 'dados/emprestimos\\emprestimos-20141.csv', 'dados/emprestimos\\emprestimos-20142.csv', 'dados/emprestimos\\emprestimos-20151.csv', 'dados/emprestimos\\emprestimos-20152.csv', 'dados/emprestimos\\emprestimos-20161.csv', 'dados/emprestimos\\emprestimos-20162.csv', 'dados/emprestimos\\emprestimos-20171.csv', 'dados/emprestimos\\emprestimos-20172.csv', 'dados/emprestimos\\emprestimos-20181.csv', 'dados/emprestimos\\emprestimos-20182.csv', 'dados/emprestimos\\emprestimos-20191.csv', 'dados/emprestimos\\emprestimos-20192.csv', 'dados/emprestimos\\emprestimos-20201.csv']


### Checando se a coluna `data_emprestimo` existe e se tem datas válidas nos arquivos .csv

Verificação intermediária inserida por conta da necessidade do `Desafio03`.

In [55]:
print("\n🔍 INICIANDO VERIFICAÇÃO DOS ARQUIVOS .CSV\n")

for caminho in caminhos_csv:
    print(f'\n📄 Verificando o arquivo: {caminho}')
    try:
        df_temp = pd.read_csv(caminho, sep=',', encoding='utf-8')

        if 'data_emprestimo' in df_temp.columns:
            df_temp['data_emprestimo'] = pd.to_datetime(df_temp['data_emprestimo'], errors='coerce')
            total = len(df_temp)
            nulos = df_temp['data_emprestimo'].isnull().sum()
            validos = total - nulos

            print(f'✔️ Total de linhas: {total}')
            print(f'🟢 Datas válidas: {validos}')
            print(f'🔴 Datas nulas/NaT: {nulos}')

            if validos > 0:
                anos = df_temp['data_emprestimo'].dropna().dt.year.value_counts().sort_index()
                print(f'📆 Anos detectados: \n{anos}')
        else:
            print('⚠️ Coluna "data_emprestimo" não encontrada.')
    
    except Exception as e:
        print(f'❌ Erro ao ler {caminho}: {e}')



🔍 INICIANDO VERIFICAÇÃO DOS ARQUIVOS .CSV


📄 Verificando o arquivo: dados/emprestimos\emprestimos-20101.csv
✔️ Total de linhas: 95357
🟢 Datas válidas: 95357
🔴 Datas nulas/NaT: 0
📆 Anos detectados: 
data_emprestimo
2010    95357
Name: count, dtype: int64

📄 Verificando o arquivo: dados/emprestimos\emprestimos-20102.csv
✔️ Total de linhas: 100696
🟢 Datas válidas: 100696
🔴 Datas nulas/NaT: 0
📆 Anos detectados: 
data_emprestimo
2010    100696
Name: count, dtype: int64

📄 Verificando o arquivo: dados/emprestimos\emprestimos-20111.csv
✔️ Total de linhas: 108547
🟢 Datas válidas: 108547
🔴 Datas nulas/NaT: 0
📆 Anos detectados: 
data_emprestimo
2011    108547
Name: count, dtype: int64

📄 Verificando o arquivo: dados/emprestimos\emprestimos-20112.csv
✔️ Total de linhas: 120813
🟢 Datas válidas: 120813
🔴 Datas nulas/NaT: 0
📆 Anos detectados: 
data_emprestimo
2011    120813
Name: count, dtype: int64

📄 Verificando o arquivo: dados/emprestimos\emprestimos-20121.csv
✔️ Total de linhas: 132606
🟢 Data

### Criando um DataFrame de DataFrames e Concatenando

In [70]:
# Adicionando todos os CSVs em um único DataFrame

dataframes = []

for caminho in caminhos_csv: 
    print(f'Lendo o arquivo: {caminho}') 
    try:
        # Forçando leitura de todas as colunas como string para evitar erros com datas e floats
        df = pd.read_csv(caminho, sep=',', encoding='utf-8', dtype=str) 
        print(f'Primeiras linhas do arquivo {caminho}:')
        print(df.head()) 
        dataframes.append(df)
    except Exception as e:
        print(f'Erro ao ler {caminho}: {e}')

if not dataframes:
    print('Nenhum arquivo foi lido corretamente!')
else:
    df_emprestimos = pd.concat(dataframes, ignore_index=True)
    print("DataFrame final concatenado:")
    print(df_emprestimos.head())


Lendo o arquivo: dados/emprestimos\emprestimos-20101.csv
Primeiras linhas do arquivo dados/emprestimos\emprestimos-20101.csv:
  id_emprestimo codigo_barras                 data_renovacao  \
0        709684       L095049                            NaN   
1        709685       L167050                            NaN   
2        709686    2006017618  2010/01/26 08:07:01.738000000   
3        709687       L184117  2010/01/18 11:07:46.470000000   
4        709684       L095049                            NaN   

                 data_emprestimo                 data_devolucao  \
0  2010/01/04 07:44:10.721000000  2010/01/05 16:26:12.662000000   
1  2010/01/04 07:44:10.750000000  2010/01/12 07:34:13.934000000   
2  2010/01/04 08:08:44.081000000  2010/02/25 07:36:25.800000000   
3  2010/01/04 08:24:21.284000000  2010/02/03 08:58:45.692000000   
4  2010/01/04 07:44:10.721000000  2010/01/05 16:26:12.662000000   

  matricula_ou_siape    tipo_vinculo_usuario  
0       2008023265.0      ALUNO DE GRAD

In [71]:
df_emprestimos.head(3)

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario
0,709684,L095049,NaN,2010/01/04 07:44:10.721000000,2010/01/05 16:26:12.662000000,2008023265.0,ALUNO DE GRADUAÇÃO
1,709685,L167050,NaN,2010/01/04 07:44:10.750000000,2010/01/12 07:34:13.934000000,2008023265.0,ALUNO DE GRADUAÇÃO
2,709686,2006017618,2010/01/26 08:07:01.738000000,2010/01/04 08:08:44.081000000,2010/02/25 07:36:25.800000000,2008112114.0,ALUNO DE PÓS-GRADUAÇÃO


### Convertendo as colunas contendo datas para `datetime`

In [72]:
# Convertendo padrão das colunas de datas

df_emprestimos['data_emprestimo'] = pd.to_datetime(df_emprestimos['data_emprestimo'], errors='coerce')
df_emprestimos['data_devolucao'] = pd.to_datetime(df_emprestimos['data_devolucao'], errors='coerce')
df_emprestimos['data_renovacao'] = pd.to_datetime(df_emprestimos['data_renovacao'], errors='coerce')

In [73]:
df_emprestimos.head(3)

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario
0,709684,L095049,NaT,2010-01-04 07:44:10.721,2010-01-05 16:26:12.662,2008023265.0,ALUNO DE GRADUAÇÃO
1,709685,L167050,NaT,2010-01-04 07:44:10.750,2010-01-12 07:34:13.934,2008023265.0,ALUNO DE GRADUAÇÃO
2,709686,2006017618,2010-01-26 08:07:01.738,2010-01-04 08:08:44.081,2010-02-25 07:36:25.800,2008112114.0,ALUNO DE PÓS-GRADUAÇÃO


In [74]:
df_emprestimos.columns

Index(['id_emprestimo', 'codigo_barras', 'data_renovacao', 'data_emprestimo',
       'data_devolucao', 'matricula_ou_siape', 'tipo_vinculo_usuario'],
      dtype='object')

### Verificando valores nulos

In [77]:
# Verificando e tratando valores nulos

df_emprestimos.isnull().sum()

id_emprestimo                 0
codigo_barras                 0
data_renovacao          1285720
data_emprestimo               0
data_devolucao             6471
matricula_ou_siape         3170
tipo_vinculo_usuario          0
dtype: int64

### Verificando Duplicatas

In [78]:
df_emprestimos['data_emprestimo'].value_counts()

data_emprestimo
2010-01-04 09:58:46.522    7
2010-01-04 09:58:46.549    7
2010-12-06 00:00:00.000    6
2010-12-15 00:00:00.000    5
2010-11-24 00:00:00.000    4
                          ..
2013-04-12 09:19:45.309    1
2013-04-12 09:19:24.655    1
2013-04-12 09:19:24.589    1
2013-04-12 09:18:27.679    1
2020-06-09 11:54:02.852    1
Name: count, Length: 2257902, dtype: int64

In [79]:
# Verificando duplicatas no DataFrame inteiro

df_emprestimos.duplicated().sum()

np.int64(37)

In [80]:
df_emprestimos.value_counts()

id_emprestimo  codigo_barras  data_renovacao           data_emprestimo          data_devolucao           matricula_ou_siape  tipo_vinculo_usuario  
709704         2006021442     2010-01-17 18:56:43.923  2010-01-04 09:58:46.522  2010-02-01 13:59:02.528  2008009890.0        ALUNO DE GRADUAÇÃO        7
709705         2009032168     2010-01-17 18:56:43.909  2010-01-04 09:58:46.549  2010-01-20 13:14:34.464  2008009890.0        ALUNO DE GRADUAÇÃO        7
709698         2009047725     2010-01-18 14:44:41.163  2010-01-04 09:21:19.099  2010-02-02 12:02:38.444  2009046838.0        ALUNO DE GRADUAÇÃO        3
709686         2006017618     2010-01-26 08:07:01.738  2010-01-04 08:08:44.081  2010-02-25 07:36:25.800  2008112114.0        ALUNO DE PÓS-GRADUAÇÃO    2
2351391        L195470        2016-01-17 16:03:38.760  2016-01-02 10:12:01.462  2016-01-27 16:37:55.000  2014036205.0        ALUNO DE GRADUAÇÃO        2
                                                                                       

In [81]:
# Mostra todas as linhas duplicadas

df_emprestimos[df_emprestimos.duplicated()]

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario
4,709684,L095049,NaT,2010-01-04 07:44:10.721,2010-01-05 16:26:12.662,2008023265.0,ALUNO DE GRADUAÇÃO
5,709685,L167050,NaT,2010-01-04 07:44:10.750,2010-01-12 07:34:13.934,2008023265.0,ALUNO DE GRADUAÇÃO
6,709686,2006017618,2010-01-26 08:07:01.738,2010-01-04 08:08:44.081,2010-02-25 07:36:25.800,2008112114.0,ALUNO DE PÓS-GRADUAÇÃO
7,709687,L184117,2010-01-18 11:07:46.470,2010-01-04 08:24:21.284,2010-02-03 08:58:45.692,200721119.0,ALUNO DE GRADUAÇÃO
19,709698,2009047725,2010-01-18 14:44:41.163,2010-01-04 09:21:19.099,2010-02-02 12:02:38.444,2009046838.0,ALUNO DE GRADUAÇÃO
20,709698,2009047725,2010-01-18 14:44:41.163,2010-01-04 09:21:19.099,2010-02-02 12:02:38.444,2009046838.0,ALUNO DE GRADUAÇÃO
28,709704,2006021442,2010-01-17 18:56:43.923,2010-01-04 09:58:46.522,2010-02-01 13:59:02.528,2008009890.0,ALUNO DE GRADUAÇÃO
29,709705,2009032168,2010-01-17 18:56:43.909,2010-01-04 09:58:46.549,2010-01-20 13:14:34.464,2008009890.0,ALUNO DE GRADUAÇÃO
30,709704,2006021442,2010-01-17 18:56:43.923,2010-01-04 09:58:46.522,2010-02-01 13:59:02.528,2008009890.0,ALUNO DE GRADUAÇÃO
31,709705,2009032168,2010-01-17 18:56:43.909,2010-01-04 09:58:46.549,2010-01-20 13:14:34.464,2008009890.0,ALUNO DE GRADUAÇÃO


In [82]:
# Garantindo que as linhas são realmente duplicadas

df_emprestimos[df_emprestimos.duplicated(keep=False)]

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario
0,709684,L095049,NaT,2010-01-04 07:44:10.721,2010-01-05 16:26:12.662,2008023265.0,ALUNO DE GRADUAÇÃO
1,709685,L167050,NaT,2010-01-04 07:44:10.750,2010-01-12 07:34:13.934,2008023265.0,ALUNO DE GRADUAÇÃO
2,709686,2006017618,2010-01-26 08:07:01.738,2010-01-04 08:08:44.081,2010-02-25 07:36:25.800,2008112114.0,ALUNO DE PÓS-GRADUAÇÃO
3,709687,L184117,2010-01-18 11:07:46.470,2010-01-04 08:24:21.284,2010-02-03 08:58:45.692,200721119.0,ALUNO DE GRADUAÇÃO
4,709684,L095049,NaT,2010-01-04 07:44:10.721,2010-01-05 16:26:12.662,2008023265.0,ALUNO DE GRADUAÇÃO
...,...,...,...,...,...,...,...
1504570,2461095,2014007744,NaT,2016-06-14 11:16:34.459,2016-06-22 08:04:39.000,20160126401.0,ALUNO DE GRADUAÇÃO
1504571,2461096,2016005538,NaT,2016-06-14 11:16:58.596,2016-07-04 16:07:41.000,20160138959.0,ALUNO DE GRADUAÇÃO
1504572,2461097,2014083427,NaT,2016-06-14 11:17:29.152,2016-06-16 09:17:19.000,20160136918.0,ALUNO DE GRADUAÇÃO
1504573,2461098,2014049460,NaT,2016-06-14 11:18:59.254,2016-06-26 13:44:13.000,20170053372.0,ALUNO DE GRADUAÇÃO


### Excluindo duplicatas

In [83]:
df_emprestimos = df_emprestimos.drop_duplicates()
df_emprestimos.value_counts()

id_emprestimo  codigo_barras  data_renovacao           data_emprestimo          data_devolucao       matricula_ou_siape  tipo_vinculo_usuario  
10000000       2015036976     2017-08-29 20:19:14.111  2017-08-16 16:21:35.442  2017-09-27 15:04:18  2012937973.0        ALUNO DE GRADUAÇÃO        1
2651446        2015035139     2018-05-17 13:18:19.350  2018-05-02 12:38:19.859  2018-06-01 12:51:09  20180050472.0       ALUNO DE GRADUAÇÃO        1
2651421        2013006520     2018-05-15 13:39:43.549  2018-05-02 12:25:21.926  2018-05-23 13:42:56  20180039312.0       ALUNO DE GRADUAÇÃO        1
2651422        2013006495     2018-05-16 06:55:17.556  2018-05-02 12:25:42.187  2018-06-11 11:37:46  20180038790.0       ALUNO DE GRADUAÇÃO        1
2651423        2014079547     2018-05-16 06:55:17.520  2018-05-02 12:25:42.222  2018-05-23 16:52:26  20180038790.0       ALUNO DE GRADUAÇÃO        1
                                                                                                               

In [84]:
# Verificando duplicatas no DataFrame inteiro

df_emprestimos.duplicated().sum()

np.int64(0)

In [85]:
# Mostra todas as linhas duplicadas

df_emprestimos[df_emprestimos.duplicated()]

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario


### Últimos ajustes no DataFrame de empréstimos

In [86]:
# Verificando os tipos de dados do DataFrame

df_emprestimos.dtypes

id_emprestimo                   object
codigo_barras                   object
data_renovacao          datetime64[ns]
data_emprestimo         datetime64[ns]
data_devolucao          datetime64[ns]
matricula_ou_siape              object
tipo_vinculo_usuario            object
dtype: object

In [87]:
# Verificando valores nulos na coluna código de barras, que será a relação com o arquivo dados do acervo

df_emprestimos['codigo_barras'].isnull().sum()

np.int64(0)

### Importando os dados do acervo

In [89]:
# Carregando o arquivo de dados do acervo

df_acervo = pd.read_parquet('dados/exemplares/dados_exemplares.parquet')
df_acervo.head()

,id_exemplar,codigo_barras,colecao,biblioteca,status_material,localizacao,registro_sistema
index,,,,,,,
0,5,L000003,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,694,1
1,4,L000002,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,688,1
2,3,L000001,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,638,1
3,7,L000114,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,616,5
5,10,L000041,Acervo Circulante,Biblioteca Central Zila Mamede,ESPECIAL,657,15


### Validações do DataFrame de acervo

In [90]:
# Verificações do arquivo de dados do acervo

df_acervo.info()

<class 'pandas.core.frame.DataFrame'>
Index: 546237 entries, 0 to 568584
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   id_exemplar       546237 non-null  int64 
 1   codigo_barras     546237 non-null  object
 2   colecao           546237 non-null  object
 3   biblioteca        546237 non-null  object
 4   status_material   546237 non-null  object
 5   localizacao       546237 non-null  int64 
 6   registro_sistema  546237 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 33.3+ MB


In [91]:
# Verifica valores nulos

df_acervo.isnull().sum()

id_exemplar         0
codigo_barras       0
colecao             0
biblioteca          0
status_material     0
localizacao         0
registro_sistema    0
dtype: int64

In [92]:
# verifica colunas únicas ou  irrelevantes

df_acervo.nunique()

id_exemplar         546141
codigo_barras       545049
colecao                 18
biblioteca              25
status_material          3
localizacao            900
registro_sistema    198503
dtype: int64

In [93]:
# Visualiza amostras aleatórias

df_acervo.sample(5)

,id_exemplar,codigo_barras,colecao,biblioteca,status_material,localizacao,registro_sistema
index,,,,,,,
204227,1295549,2018010083,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,639,85129
414892,1038189,2013011464,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,316,190655
246898,248193,2008011296,Folhetos,Biblioteca Central Zila Mamede,ESPECIAL,698,104442
563664,1340837,2019028390,Publicações de Autores do RN,Biblioteca Central Zila Mamede,NÃO CIRCULA,898,263728
184499,193400,2006041155,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,664,74113


In [94]:
# Verifica o tipo de dados

df_acervo.dtypes

id_exemplar          int64
codigo_barras       object
colecao             object
biblioteca          object
status_material     object
localizacao          int64
registro_sistema     int64
dtype: object

### Unindo os dois DataFrames

In [95]:
# Mesclando os DataFrames de Emprestimos e Acervo

# O método merge, irá mesclar os DataFrames, usando como referência a coluna 'codigo_barras'
df_completo = pd.merge(df_emprestimos, # DataFrame da esquerda
                       df_acervo, # DataFrame da direita
                       on = 'codigo_barras', # Coluna em comum entre os dois
                       how = 'left' # Tipo de junção (LEFT JOIN)
                       )
# how = left significa: "traga todos os registros do DataFrame da esquerda (df_emprestimos) e complemente com os dados do da direita (df_acervo) quando houver correspondência"

df_completo.head()

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao,registro_sistema
0,709684,L095049,NaT,2010-01-04 07:44:10.721,2010-01-05 16:26:12.662,2008023265.0,ALUNO DE GRADUAÇÃO,13259.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,630.0,4225.0
1,709685,L167050,NaT,2010-01-04 07:44:10.750,2010-01-12 07:34:13.934,2008023265.0,ALUNO DE GRADUAÇÃO,70865.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,647.0,25009.0
2,709686,2006017618,2010-01-26 08:07:01.738,2010-01-04 08:08:44.081,2010-02-25 07:36:25.800,2008112114.0,ALUNO DE PÓS-GRADUAÇÃO,195347.0,Acervo Circulante,Biblioteca Setorial Prof. Rodolfo Helinski - E...,REGULAR,640.0,75019.0
3,709687,L184117,2010-01-18 11:07:46.470,2010-01-04 08:24:21.284,2010-02-03 08:58:45.692,200721119.0,ALUNO DE GRADUAÇÃO,131639.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,609.0,44494.0
4,709688,2007027500,2010-01-19 15:59:26.464,2010-01-04 08:27:33.569,2010-02-03 13:06:30.662,2009045971.0,ALUNO DE GRADUAÇÃO,225646.0,Acervo Circulante,Biblioteca Central Zila Mamede,REGULAR,686.0,91736.0


### Verificando se há linhas sem correspondência e o percentual

In [96]:
# Verificando quantas linhas não encontraram correspondência no acervo

faltando_acervo = df_completo['id_exemplar'].isnull().sum()
print(f'Linhas sem correspondência no acervo: {faltando_acervo}')

percent_faltando = (faltando_acervo / len(df_completo)) * 100
print(f'{percent_faltando:.2f}% dos empréstimos não têm dados de acervo.')

Linhas sem correspondência no acervo: 189257
8.37% dos empréstimos não têm dados de acervo.


### Verificando quais os códigos de barra que não encontraram correspondência

In [97]:
# Verificando quais códigos de barras não encontraram correspondência

# Seleciona a coluna 'id_exemplar' que vem de df_acervo. Se ela tiver NaN após o merge, não houve correspondência para 
# Aquele código de barras.
# isna() retorna True para valores NaN, e False para valores preenchidos
# df_sem_match está recebendo os emprestimos que não tiveram correspondência com os ID's de exemplares do acervo.
df_sem_match = df_completo[df_completo['id_exemplar'].isna()]
df_sem_match.head()

,id_emprestimo,codigo_barras,data_renovacao,data_emprestimo,data_devolucao,matricula_ou_siape,tipo_vinculo_usuario,id_exemplar,colecao,biblioteca,status_material,localizacao,registro_sistema
17,709701,L163841,2010-01-19 19:02:53.450,2010-01-04 09:48:05.604,2010-02-04 18:23:49.946,2008029450.0,ALUNO DE GRADUAÇÃO,NaN,NaN,NaN,NaN,NaN,NaN
98,709782,L165850,NaT,2010-01-04 14:06:16.699,2010-01-22 16:09:20.065,200621343.0,ALUNO DE GRADUAÇÃO,NaN,NaN,NaN,NaN,NaN,NaN
109,709793,L178388,2010-01-19 16:31:44.491,2010-01-04 14:47:41.976,2010-02-01 16:29:17.951,200506323.0,ALUNO DE GRADUAÇÃO,NaN,NaN,NaN,NaN,NaN,NaN
118,709802,2009032201,2010-01-19 16:13:22.082,2010-01-04 14:57:23.766,2010-02-03 14:04:04.222,2009053966.0,ALUNO DE GRADUAÇÃO,NaN,NaN,NaN,NaN,NaN,NaN
119,709803,L188583,2010-02-01 15:52:28.614,2010-01-04 15:03:04.753,2010-03-02 17:58:07.985,2010116854.0,ALUNO DE PÓS-GRADUAÇÃO,NaN,NaN,NaN,NaN,NaN,NaN


In [98]:
# Retorna os valores únicos da coluna código de barras
# Quais foram os códigos de barras que não foram encontrados no acervo?
df_sem_match['codigo_barras'].unique()

array(['L163841', 'L165850', 'L178388', ..., 'L143612', '2018007270',
       'Y021773'], shape=(38540,), dtype=object)

### Exportando o DataFrame Completo para ser usado futuramente

In [100]:
df_completo.to_csv('../desafio02/dataset_completo.csv', index = False)